In [2]:
from scipy.spatial.distance import hamming
import numpy as np
import matplotlib.pyplot as plt
import time
import torch
import torch.nn.functional as F
import math
from Module import ReconstructionTool 

In [3]:
start_time = time.time()
np.random.seed(42)
torch.manual_seed(42)
# Set individuals count for analysis
individuals = [50]
snp_count = 30

In [4]:
# Arrays to store results
precisions = []
recalls = []
f1_scores = []
accuracies = []
# Load data files
beacon = np.load("beacon.npy")
print("Loaded beacon data...")
print(f"Beacon data loaded. Shape: {beacon.shape}")


FileNotFoundError: [Errno 2] No such file or directory: 'beacon.npy'

In [46]:
for ind_count in individuals:
    print("Individual count:", ind_count)
    print("-" * 80)

    # Load and slice the initial correlation matrix
    print("Loading correlation matrix...")
    corr1 = np.load("OpenSNP.npy")
    corr1 = corr1[:snp_count , :100]

    # Calculate correlations using the class
    corr_tool = ReconstructionTool(corr1)
    corr = corr_tool.calculate_correlations(corr1)
    print(corr.shape, "Initial Corr")

    # Create tensor of correlations
    corr = torch.tensor(corr, dtype=torch.float32, requires_grad=True)

    # Slice the beacon for the first `ind_count` individuals
    beacon = beacon[:snp_count , :ind_count]
    print(f"Beacon partitioning for {ind_count} individuals. Shape: {beacon.shape}")
    beacon_tool = ReconstructionTool(beacon)
    print(beacon.shape)
    number_of_ones = np.sum(beacon)
    print("Number of ones in beacon:", number_of_ones)
    print(beacon, "Initial Beacon")

    # Query and reconstruct beacon
    reconstructed_beacon = beacon_tool.query(proportion=1.0)
    print(reconstructed_beacon, "Initial Reconstructed Beacon")
    print("Number of ones in reconstructed beacon:", number_of_ones)
    reconstructed_beacon = beacon_tool.compare_and_sort_columns(beacon, reconstructed_beacon)
    
    number_of_ones = np.sum(reconstructed_beacon)
    

    



Individual count: 50
--------------------------------------------------------------------------------
Loading correlation matrix...
(30, 30) Initial Corr
Beacon partitioning for 50 individuals. Shape: (30, 50)
(30, 50)
Number of ones in beacon: 351.0
[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 1. 1. ... 1. 0. 0.]] Initial Beacon
[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 1. 1. ... 1. 0. 0.]] Initial Reconstructed Beacon
Number of ones in reconstructed beacon: 351.0


In [47]:
# Training loop setup
EPOCHS = 1001
EPOCHS1 = 501
max_iterations = 3000
iteration = 0
converged = False
target_frequencies = torch.tensor(np.sum(beacon, axis=1), dtype=torch.float32)
prev_reconstructed_beacon = None  


while iteration < max_iterations:
    iteration += 1
        # Stage 1: Optimize correlations
    reconstructed_beacon_tensor = torch.tensor(reconstructed_beacon, dtype=torch.float32, requires_grad=True)
    optimizer = torch.optim.Adam([reconstructed_beacon_tensor], lr=0.001)

    for epoch in range(EPOCHS):
        optimizer.zero_grad()
        loss = torch.norm(corr_tool.calculate_correlations(reconstructed_beacon_tensor) - corr, p='fro')
        loss.backward()
        optimizer.step()
        # if epoch % 100 == 0:
        #     print(f'Iteration {iteration}, Epoch {epoch}, loss {loss.item()}')

    # print(reconstructed_beacon, "Reconstructed Beacon")

        # Stage 2: Optimize for SNP frequencies
    reconstructed_beacon_tensor = torch.tensor(reconstructed_beacon, dtype=torch.float32, requires_grad=True)
    optimizer = torch.optim.Adam([reconstructed_beacon_tensor], lr=0.001)

    for epoch in range(EPOCHS1):
        optimizer.zero_grad()
        freq_loss = beacon_tool.frequency_loss(reconstructed_beacon_tensor, target_frequencies)
        freq_loss.backward()
        optimizer.step()

        # if epoch % 100 == 0:
        #     print(f'Iteration {iteration}, Epoch {epoch}, Frequency loss: {freq_loss.item()}')

    reconstructed_beacon = (reconstructed_beacon_tensor.detach().numpy() > 0.5).astype(int)
    # print(reconstructed_beacon, "Frequency-Optimized Reconstructed Beacon")

    # Convergence check based on flips
    if prev_reconstructed_beacon is not None:
        # Count the number of flips between current and previous beacons
        flips = np.sum(reconstructed_beacon != prev_reconstructed_beacon)
        

        # Check if flips are less than 10
        if flips < 10:
            converged = True
            break

    # Update previous beacon
    prev_reconstructed_beacon = reconstructed_beacon.copy()



new_beacon = beacon_tool.compare_and_sort_columns(beacon, reconstructed_beacon)
print(new_beacon, "Sorted Reconstructed Beacon")

[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]] Sorted Reconstructed Beacon


In [48]:
# Compute metrics
accuracy = beacon_tool.calculate_accuracy(beacon, new_beacon)
precision = beacon_tool.calculate_precision(beacon, new_beacon)
recall = beacon_tool.calculate_recall(beacon, new_beacon)
f1 = beacon_tool.calculate_f1(beacon, new_beacon)

accuracies.append(accuracy)
precisions.append(precision)
recalls.append(recall)
f1_scores.append(f1)

In [49]:
#print the results in a table format, with .3f precision, save
print("Individuals\tAccuracy\tPrecision\tRecall\t\tF1 Score")
# print the result of the initial beacon

for i in range(len(individuals)):
    print(f"{individuals[i]}\t\t{accuracies[i]:.3f}\t\t{precisions[i]:.3f}\t\t{recalls[i]:.3f}\t\t{f1_scores[i]:.3f}")

Individuals	Accuracy	Precision	Recall		F1 Score
50		0.901		0.801		0.766		0.783
